# 通过 TimedRotatingFileHandler 按时间切割日志

> 线上跑了一个定时脚本，每天生成的日志文件都写在了一个文件中。但是日志信息不可能输出到单一的一个文件中。  
> 原因有二：1. 日志文件越来越大会影响系统的性能。2. 日志文件格式不够清晰，比如我想看今天的日志，不太方便找到的今天的日志信息 (即使对日志输出做了时间提示)  
> 通过设置`TimedRotatingFileHandler`进行日志按**周 (W)、天(D)、时(H)、分(M)、秒(S)** 切割。  

先看一个简单例子：

```py
import time
import logging
import os
from logging import handlers


def _logging(**kwargs):
    level = kwargs.pop('level', None)
    filename = kwargs.pop('filename', None)
    datefmt = kwargs.pop('datefmt', None)
    format = kwargs.pop('format', None)
    if level is None:
        level = logging.DEBUG
    if filename is None:
        filename = 'default.log'
    if datefmt is None:
        datefmt = '%Y-%m-%d %H:%M:%S'
    if format is None:
        format = '%(asctime)s [%(module)s] %(levelname)s [%(lineno)d] %(message)s'

    log = logging.getLogger(filename)
    format_str = logging.Formatter(format, datefmt)
    # backupCount 保存日志的数量，过期自动删除
    # when 按什么日期格式切分(这里方便测试使用的秒)
    th = handlers.TimedRotatingFileHandler(filename=filename, when='S', backupCount=3, encoding='utf-8')
    th.setFormatter(format_str)
    th.setLevel(logging.INFO)
    log.addHandler(th)
    log.setLevel(level)
    
    # 为了看的更视觉效果，可以显示在控制台答应
    cmd = logging.StreamHandler()
    cmd.setFormatter(format_str)
    cmd.setLevel(level)
    log.addHandler(cmd)
    
    return log


os.makedirs("./logs", exist_ok=True)
logger = _logging(filename='./logs/default.log')

if __name__ == '__main__':
    while True:
        time.sleep(0.1)
        logger.info('哈哈哈')
```

结果如下：  
![](../assets/logging1.png)

## 参数

- filename：日志文件名的prefix；
- when：是一个字符串，用于描述滚动周期的基本单位，字符串的值及意义如下：
“S”: Seconds  
“M”: Minutes  
“H”: Hours  
“D”: Days  
“W”: Week day (0=Monday)  
“midnight”: Roll over at midnight  
- interval: 滚动周期，单位有when指定，比如：when=’D’,interval=1，表示每天产生一个日志文件；
- backupCount: 表示日志文件的保留个数；


## 自定义日志文件名

> 上述代码可以正常运行，而且也可以生成固定的日志个数，但是有一个问题，生成的日志文件格式是你的`文件名+时间`的格式，没有设置时间的话默认设置到了秒 (这里是按秒切割)  

修改日志文件名格式

### 自定义日期后缀格式

除了上述参数之外，`TimedRotatingFileHandler`还有两个比较重要的成员变量，它们分别是`suffix`和`extMatch`。`suffix`是指日志文件名的后缀,`suffix`中通常带有格式化的时间字符串，filename和suffix由“.”连接构成文件名（例如：`filename=“runtime”， suffix=“%Y-%m-%d.log”`,生成的文件名为runtime.2015-07-06.log）。

```py
# 设置为S，默认的suffix为 Y-%m-%d_%H-%M-%S
filehandler.suffix = "%Y-%m-%d_%H-%M-%S.log"
```

### 自定义整个日志文件名格式（❗有坑、慎用）

```py
# 在上述代码中加入一个函数：
def namer(filename):
    # filename参数为日志文件的绝对路径
    # 返回重命名后的文件绝对路径
    return filename.replace('default.log.', '')
```

然后给handler的namer 属性赋值:

```py
filehandler.namer = namer
```

运行结果：  
![](../assets/logging2.png)

> 名字好像可以了，**但是日志好像没有起到自动删除的目的**, 来看看源码：

```py
def getFilesToDelete(self):
        """
        Determine the files to delete when rolling over.

        More specific than the earlier method, which just used glob.glob().
        """
        dirName, baseName = os.path.split(self.baseFilename)
        fileNames = os.listdir(dirName)
        result = []
        prefix = baseName + "."
        plen = len(prefix)
        for fileName in fileNames:
            if fileName[:plen] == prefix:
                suffix = fileName[plen:]
                if self.extMatch.match(suffix):
                    result.append(os.path.join(dirName, fileName))
        if len(result) < self.backupCount:
            result = []
        else:
            result.sort()
            result = result[:len(result) - self.backupCount]
        return result
```

> 这是它的删除逻辑，是通过查找`self.baseFilename`同一目录下的，带有filename（我此处为default.log）前缀的文件列表，按时间从大到小排序后取，返回大于 `backupCount` 数量的要删除的文件名列表。

由于后来修改文件名格式时把文件名前缀`default.log.`给替换为空，没有了了公共前缀，便不能删除过量的日志文件了。所以在 namer 属性定义的函数中，一定要保留baseFilename（default.log)字符串。

### 删除日志设置：extMatch属性

extMatch是一个编译好的正则表达式，用于匹配日志文件名的后缀，一般来说它和suffix是匹配的，如果extMatch匹配不到日志文件名的话，过期的日志是不会被删除的。比如，suffix=“%Y-%m-%d.log”, extMatch的只应该是re.compile(r”^\d{4}-\d{2}-\d{2}.log$”)。默认情况下，在TimedRotatingFileHandler对象初始化时，`suffix`和`extMatch`会根据when的值进行初始化。

| when     | suffix             | extMatch                                |
| :------- | :----------------- | :--------------------------------------- |
| S’       | %Y-%m-%d\_%H-%M-%S | r"\^d{4}-\d{2}-\d{2}\_\d{2}-\d{2}-\d{2}" |
| M        | %Y-%m-%d\_%H-%M    | r"^\d{4}-\d{2}-\d{2}\_\d{2}-\d{2}"       |
| H        | %Y-%m-%d\_%H       | r"^\d{4}-\d{2}-\d{2}\_\d{2}"             |
| D        | %Y-%m-%d           | r"^\d{4}-\d{2}-\d{2}"                    |
| MIDNIGHT | %Y-%m-%d           | r"^\d{4}-\d{2}-\d{2}"                    |
| W        | Y-%m-%d”           | r"^\d{4}-\d{2}-\d{2}"                    |

**如果对日志文件名没有特殊要求的话，可以不用设置suffix和extMatch**，如果需要修改了suffix，也一定要修改extMatch，让它们匹配上。

例如：

```py
filehandler.suffix = "%Y-%m-%d_%H-%M.log"
filehandler.extMatch = re.compile(r"^\d{4}-\d{2}-\d{2}_\d{2}-\d{2}.log$")
```

该属性作用见上面的`getFilesToDelete`中


---

```py
import time
import logging
import os
from logging import handlers


def _logging(**kwargs):
    level = kwargs.pop('level', None)
    filename = kwargs.pop('filename', None)
    datefmt = kwargs.pop('datefmt', None)
    format_ = kwargs.pop('format', None)
    if level is None:
        level = logging.DEBUG
    # filename 要保存的文件名
    if filename is None:
        filename = 'default.log'
    # datefmt 时间格式
    if datefmt is None:
        datefmt = '%Y-%m-%d %H:%M:%S'
    if format_ is None:
        format_ = '%(asctime)s [%(name)s:%(lineno)d] [%(levelname)s]- %(message)s'

    logger = logging.getLogger(__name__)
    logger.setLevel(level)
    formatter = logging.Formatter(format_, datefmt)

    # 1.设置log日志文件按时间拆分记录，并保存几个历史文件，如果不需要拆分文件记录可忽略
    # 例：设置每天保存一个log文件，以日期为后缀，保留3个旧文件。
    filehandler = handlers.TimedRotatingFileHandler(filename=filename, when='S', backupCount=3, encoding='utf-8')
    filehandler.suffix = "%Y-%m-%d_%H-%M-%S.log"  # 设置历史文件 后缀
    filehandler.setFormatter(formatter)
    filehandler.setLevel(logging.INFO)
    logger.addHandler(filehandler)

    # 2.设置log日志的标准输出打印，如果不需要在终端输出结果可忽略
    console = logging.StreamHandler()
    console.setFormatter(formatter)
    console.setLevel(level)
    # 只为当前logger启用console
    logger.addHandler(console)

    # 所有日志都启用console
    # logging.getLogger('').addHandler(console)

    return logger


os.makedirs("./logs", exist_ok=True)
logger = _logging(filename='./logs/default.log')

if __name__ == '__main__':
    while True:
        time.sleep(0.1)
        logger.info('哈哈哈')
```